In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json
import os
import math
import matplotlib.pyplot as plt
import pandas_datareader as web
try:
    import yfinance as yf
except:
    !pip install yfinance
    import yfinance as yf
from datetime import date, timedelta, datetime
from pandas.io.json import json_normalize
import sqlite3
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.28.2
    Uninstalling requests-2.28.2:
      Successfully uninstalled requests-2.28.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
beatrix-jupyterlab 2023.58.190319 requires jupyter-server~=1.16, but you have jupyter-server 2.5.0 which is incompatible.
gcsfs 2023.5.0 requires fsspec==2023.5.0, but you have fsspec 2023.6.0 which is incompatible.
google-cloud-artifact-registry 1.8.1 requires google-api-core[grpc]!=2.0.*,!=2.1.*,!=2.10.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,!=2.9.*,<3.0.0dev,>=1.34.0, but you have 

In [2]:
conn = sqlite3.connect('fundamental_analytics.db')

In [3]:
def get_start_date():
    print("Please enter the start date for evaluation in DD-MM-YYYY (example: 01-01-2019): ")
    start_date = input()
    start_date_object = datetime.strptime(start_date, '%d-%m-%Y')
    print("Date entered by you: " + start_date_object.strftime('%d/%m/%Y'))
    return start_date_object

def get_end_date():
    print("Please enter the end date for evaluation in DD-MM-YYYY (example: 31-10-2019): ")
    end_date = input()
    end_date_object = datetime.strptime(end_date, '%d-%m-%Y')
    print("Date entered by you: " + end_date_object.strftime('%d/%m/%Y'))
    return end_date_object

def get_stock_tickers():
    print("Please enter the list of stocks seprated by comma: ")
    get_stock_tickers_object = input().split(',')
    print("The Tickers entered by you are as follows: ")
    for ticker in range(len(get_stock_tickers_object)): 
        print(get_stock_tickers_object[ticker])
    return get_stock_tickers_object

def get_stock_ticker_data(stock_ticker, start_date, end_date):
    print("Evaluating for " + stock_ticker)
    scrip = web.DataReader(stock_ticker, data_source='yahoo', start=start_date.strftime('%d/%m/%Y'), end=end_date.strftime('%d/%m/%Y'))
    scrip['Ticker'] = stock_ticker
    return scrip

def get_all_stocks_dataframe(stock_tickers, start_date, end_date):
    df_all_historicaldata = list()
    for ticker in range(len(stock_tickers)):
        df_stock_data = get_stock_ticker_data(stock_tickers[ticker],start_date,end_date)
        df_all_historicaldata.append(df_stock_data)

    df_all_historicaldata.append(get_stock_ticker_data('^NSEI',start_date,end_date))
    df_all_historicaldata.append(get_stock_ticker_data('^CRSLDX',start_date,end_date))
    print(len(df_all_historicaldata))

    return df_all_historicaldata

def get_merged_frame(historical_value_dataframe):
    merged_dataframe_result = pd.DataFrame()
    for iterator in range(len(historical_value_dataframe)):
        if(iterator==0):
            data = historical_value_dataframe[iterator]
            scrip = 'Close_' + data['Ticker'][0]
            data = data.rename(columns = {'Close' : scrip})
            merged_dataframe_result = data[[scrip]]
        else:
            data = historical_value_dataframe[iterator]
            scrip = 'Close_' + data['Ticker'][0]
            data = data[['Close']]
            data = data.rename(columns = {'Close' : scrip})
            merged_dataframe_result = merged_dataframe_result.join(data, how='left')
    print(merged_dataframe_result.head(5))
    return merged_dataframe_result

def get_BS_DATA(ticker):
    df_BS = yf.Ticker(ticker)
    print("----- BALANCE SHEET DATA -----")
    print(df_BS.balance_sheet)
    print(type(df_BS))
    return df_BS.balance_sheet

def get_CF_DATA(ticker):
    df_CF = yf.Ticker(ticker)
    print("----- CASH FLOW -----")
    df_CF.cashflow
    return df_CF.cashflow

def get_PNL_DATA(ticker):
    df_PNL = yf.Ticker(ticker)
    print("----- PNL -----")
    df_PNL.financials
    return df_PNL.financials

In [4]:
def get_fundamental_data():
    list_of_stocks = input("Please enter multiple scrips separated by comma: ")
    final_list = list_of_stocks.split(",")
    print(final_list)
    return final_list

In [5]:
def generate_financial_dataset(ticker):
    get_BS_DATA(ticker).to_csv("DF_BS_DATA.csv")
    get_CF_DATA(ticker).to_csv("DF_CF_DATA.csv")
    get_PNL_DATA(ticker).to_csv("DF_PNL_DATA.csv")

In [6]:
generate_financial_dataset('SUNPHARMA.NS')

----- BALANCE SHEET DATA -----
                                                      2023-03-31  \
Ordinary Shares Number                              2399334970.0   
Share Issued                                        2399334970.0   
Net Debt                                           21340600000.0   
Total Debt                                         68896600000.0   
Tangible Book Value                               379558300000.0   
...                                                          ...   
Cash Cash Equivalents And Short Term Investments  159074100000.0   
Other Short Term Investments                      112836800000.0   
Cash And Cash Equivalents                          46237300000.0   
Cash Equivalents                                             NaN   
Cash Financial                                               NaN   

                                                      2022-03-31  \
Ordinary Shares Number                              2399334970.0   
Share Issued    

In [7]:
ticker = 'SUNPHARMA.NS'
df = get_BS_DATA(ticker)
df_pnl = get_PNL_DATA(ticker)
df_cf = get_CF_DATA(ticker)

----- BALANCE SHEET DATA -----
                                                      2023-03-31  \
Ordinary Shares Number                              2399334970.0   
Share Issued                                        2399334970.0   
Net Debt                                           21340600000.0   
Total Debt                                         68896600000.0   
Tangible Book Value                               379558300000.0   
...                                                          ...   
Cash Cash Equivalents And Short Term Investments  159074100000.0   
Other Short Term Investments                      112836800000.0   
Cash And Cash Equivalents                          46237300000.0   
Cash Equivalents                                             NaN   
Cash Financial                                               NaN   

                                                      2022-03-31  \
Ordinary Shares Number                              2399334970.0   
Share Issued    

In [8]:
df.loc['Share Issued'][0]

2399334970.0

In [9]:
def ratio(num,den):
    val=0.0
    try:
        val = round (num / den,4)
    except:
        print("!! Error for ",num," & ",den)
    finally:
        return val

def per_calc(y,y_1):
    val = 0.0
    try:
        val = round ((y_1/y)-1,4)
    except:
        print("!! Error for ",y," & ",y-1)
    finally:
        return val
    
def print_ms(var,msg):
    print(msg)
    print("\n")
    print(var)
    print("\n")

In [10]:
lst_liquidity_points = []

# Project Todo List
1. Create a duplicate copy of index columns called description.
2. Liquidity Ratios:
    * Current Ratio => Current Assets / Current Liabilities
    * Quick Ratio => (Current Assets - Inventory) / Current Liabilities
    * Absolute Liquidity => Cash + Marketable Securities + Net Receivables + Debtors
    * Cash Ratio => (Cash + Marketable Securities) / Current Liability

In [11]:
def ratio_scoring_yoy_asc(num_row_lst,den_row_lst,name):
    # current assets change list
    lst_cr = [ratio(num_row_lst[0],den_row_lst[0]),\
              ratio(num_row_lst[1],den_row_lst[1]),\
              ratio(num_row_lst[2],den_row_lst[2]),\
              ratio(num_row_lst[3],den_row_lst[3])
             ]
    lst_cr_mv = [per_calc(lst_cr[0],lst_cr[1]),per_calc(lst_cr[1],lst_cr[2]),per_calc(lst_cr[2],lst_cr[3])]
    lst_points = [1 if x>0 else 0 for x in lst_cr_mv]
    print_ms(lst_cr,"-- Ratios for "+name+" --")
    print_ms(lst_cr_mv, "-- Ratio movement for "+name+" --")
    print_ms(lst_points, "-- Ratio Points for "+name+" --")
    return [lst_points,lst_cr_mv,lst_cr]

In [12]:
def evaluate_score(df):
    
    lst_aggregator = []
    
    # calculation for current ratio
    lst_curr_lr = ratio_scoring_yoy_asc(df.loc['Current Assets'],df.loc['Current Liabilities'],"Current Ratio")

    # calculation of quick ratio
    lst_curr_qr = ratio_scoring_yoy_asc(df.loc['Current Assets']-df.loc['Inventory'],df.loc['Current Liabilities'],"Quick Ratio")

    # calculation of cash ratio
    lst_curr_car = ratio_scoring_yoy_asc(df.loc['Cash Cash Equivalents And Short Term Investments'],df.loc['Current Liabilities'],"Cash Ratio")

    # aggregate to the liquidity ratio list
    lst_aggregator_lr = [["Current Ratio",lst_curr_car,],["Quick Ratio",lst_curr_car,],["Cash Ratio",lst_curr_car,]]
    
    # aggregate to the main aggregator ratio
    lst_aggregator.append("Quick Ratio")
    lst_aggregator.append(lst_aggregator_lr)
    
    #TURNOVER RATIOS
    # calculation for inventory tunrover ratio
    lst_inv_tr = ratio_scoring_yoy_asc(df_pnl.loc['Reconciled Cost Of Revenue'],df.loc['Inventory'],"Inventory Turnover")
    
    # calculation for receivable turnover ratio
    lst_inv_rr = ratio_scoring_yoy_asc(df_pnl.loc['Operating Revenue'],df.loc['Accounts Receivable'],"Account Receivable Turnover")
    
    # calculation for working capital turnover ratio
    lst_wc_tr = ratio_scoring_yoy_asc(df_pnl.loc['Total Revenue'],df.loc['Working Capital'],"Working Capital Turnover")
    
    # calculation for asset turnover ratio
    lst_as_tr = ratio_scoring_yoy_asc(df_pnl.loc['Operating Revenue'],df.loc['Total Assets'],"Asset Turnover")
    
    # calculation for cash conversion ratio
    lst_cc_rt = ratio_scoring_yoy_asc(df_cf.loc['Operating Cash Flow'],df_pnl.loc['Normalized EBITDA'],"Cash conversion ratio")
    
    # calculation for capital turnover ratio
    lst_cp_tr = ratio_scoring_yoy_asc(df_pnl.loc['Total Revenue'],df.loc['Invested Capital'],"Cash conversion ratio")
    
    lst_aggregator_tr = [["Inv_Turnover_Ratio",lst_inv_tr]
                         ,["Receivable_TR",lst_inv_tr]
                         ,["WC_TR",lst_wc_tr]
                         ,["Asset_TR",lst_as_tr]
                         ,["Cash_Conv_Ratio",lst_cc_rt]
                         ,["Capital_TR",lst_cp_tr]]
    
    #  aggregate to the main aggregator ratio
    lst_aggregator.append("Turnover Ratios")
    lst_aggregator.append(lst_aggregator_tr)
    return lst_aggregator

# Turnover Ratio
* Inventory turnover ratio
* Receivable turnover ratio
* Capital ratio
* Asset turnover ratio
* Net working capital ratio
* Cash conversion ratio

In [13]:
print(evaluate_score(df))

-- Ratios for Current Ratio --


[2.0025, 2.0357, 1.8855, 2.0154]


-- Ratio movement for Current Ratio --


[0.0166, -0.0738, 0.0689]


-- Ratio Points for Current Ratio --


[1, 0, 1]


-- Ratios for Quick Ratio --


[1.4743, 1.5126, 1.3282, 1.514]


-- Ratio movement for Quick Ratio --


[0.026, -0.1219, 0.1399]


-- Ratio Points for Quick Ratio --


[1, 0, 1]


-- Ratios for Cash Ratio --


[0.7991, 0.7357, 0.5923, 0.6965]


-- Ratio movement for Cash Ratio --


[-0.0793, -0.1949, 0.1759]


-- Ratio Points for Cash Ratio --


[0, 0, 1]


-- Ratios for Inventory Turnover --


[1.0142, 1.3013, 1.0987, 1.3297]


-- Ratio movement for Inventory Turnover --


[0.2831, -0.1557, 0.2102]


-- Ratio Points for Inventory Turnover --


[1, 0, 1]


-- Ratios for Account Receivable Turnover --


[3.7836, 3.6276, 3.6572, 3.4311]


-- Ratio movement for Account Receivable Turnover --


[-0.0412, 0.0082, -0.0618]


-- Ratio Points for Account Receivable Turnover --


[0, 1, 0]


-- Ratios for Work